In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Head_and_Neck_Cancer"
cohort = "GSE184944"

# Input paths
in_trait_dir = "../../input/GEO/Head_and_Neck_Cancer"
in_cohort_dir = "../../input/GEO/Head_and_Neck_Cancer/GSE184944"

# Output paths
out_data_file = "../../output/preprocess/Head_and_Neck_Cancer/GSE184944.csv"
out_gene_data_file = "../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE184944.csv"
out_clinical_data_file = "../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE184944.csv"
json_path = "../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Immune gene expression analysis for oral leukplakia samples"
!Series_summary	"Oral leukoplakia is common and may in some cases progress to carcinoma. Proliferative leukoplakia (PL) is a progressive, often multifocal subtype with a high rate of malignant transformation (MT) compared to the more common localized leukoplakia (LL). We hypothesized that the immune microenvironment and gene expression patterns would be distinct for PL compared to LL. We summarize key clinicopathologic features among PL and LL and compare cancer-free survival (CFS) between subgroups. We analyze immunologic gene expression profiling (GEP) in PL and LL tissue samples (NanoString PanCancer Immune Oncology Profiling). We integrate immune cell activation and spatial distribution patterns in tissue samples using multiplexed immunofluorescence and digital image capture to further define PL and LL. Among N=58 patients (PL: 29, LL: 29), only the clinical diagnosis of PL was assoc

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset appears to be gene expression data from the NanoString PanCancer Immune Oncology Profiling,
# which focuses on immune gene expression in oral leukoplakia samples
is_gene_available = True

# 2.1 Data Availability

# Trait is available in row 0, which indicates histology type (EL, LL, PEL, or PL)
# Based on the background information, PL (Proliferative leukoplakia) has higher rate of 
# malignant transformation than LL (localized leukoplakia)
trait_row = 0

# Age is not available in the sample characteristics
age_row = None

# Gender is available in row 2
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value):
    """
    Convert histology type to binary: 1 for PL (higher risk) and 0 for others
    """
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # From background info: PL has higher malignant transformation rate
    if value == "PL":  # Proliferative leukoplakia - high risk
        return 1
    elif value in ["LL", "EL", "PEL"]:  # Other types - lower risk
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age to continuous value
    """
    # Age data not available
    return None

def convert_gender(value):
    """
    Convert gender to binary: 0 for female, 1 for male
    """
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == "F":
        return 0
    elif value.upper() == "M":
        return 1
    else:
        return None

# 3. Save Metadata
# Since trait_row is not None, trait data is available
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=(trait_row is not None)
)

# Note: We're stopping here as we don't have access to the actual clinical data matrix 
# needed to perform step 4 (Clinical Feature Extraction). We have identified the rows and
# conversion functions that would be used if we had the data.
print("Analysis completed: This dataset contains gene expression data and trait information (histology types).")
print("The trait is binary with PL (Proliferative leukoplakia) representing high risk (1) and other types as lower risk (0).")
print("Gender information is available but age information is not available in this dataset.")


Analysis completed: This dataset contains gene expression data and trait information (histology types).
The trait is binary with PL (Proliferative leukoplakia) representing high risk (1) and other types as lower risk (0).
Gender information is available but age information is not available in this dataset.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Head_and_Neck_Cancer/GSE184944/GSE184944_series_matrix.txt.gz
Gene data shape: (730, 49)
First 20 gene/probe identifiers:
Index(['A2M', 'ABCB1', 'ABL1', 'ADA', 'ADORA2A', 'AICDA', 'AIRE', 'AKT3',
       'ALCAM', 'AMBP', 'AMICA1', 'ANP32B', 'ANXA1', 'APOE', 'APP', 'ARG1',
       'ARG2', 'ATF1', 'ATF2', 'ATG10'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown (A2M, ABCB1, ABL1, etc.), these are already
# standard human gene symbols that don't need mapping.
# These are official gene symbols as recognized by HGNC (HUGO Gene Nomenclature Committee).

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to Randall's plaque tissue, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data from Randall's plaque tissue associated with kidney stones."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for kidney stones analysis."
    )
    
    print("Dataset is not usable for kidney stones analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (730, 49)
Gene data shape after normalization: (726, 49)
Normalized gene expression data saved to ../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE184944.csv
Original clinical data preview:
         !Sample_geo_accession         GSM5602145         GSM5602146  \
0  !Sample_characteristics_ch1      histology: EL      histology: EL   
1  !Sample_characteristics_ch1  smoking status: N  smoking status: N   
2  !Sample_characteristics_ch1          gender: M          gender: M   

          GSM5602147         GSM5602148         GSM5602149         GSM5602150  \
0      histology: EL      histology: LL      histology: LL      histology: LL   
1  smoking status: N  smoking status: N  smoking status: C  smoking status: F   
2          gender: F          gender: F          gender: M          gender: F   

          GSM5602151         GSM5602152         GSM5602153  ...  \
0      histology: LL      histology: LL      histology: LL  ...   
1  smoking stat

Linked data saved to ../../output/preprocess/Head_and_Neck_Cancer/GSE184944.csv
